In [ ]:
# Create regions of interest version 1
# Uses hand-drawn centerlines extended into some industrial areas manually

import os
import utils.epsql as epsql
import sqlalchemy
import numpy as np

engine = epsql.Engine()


In [ ]:
blocks2010 = "tiger_wgs84.tl_2010_tabblock10"
engine.execute_returning_gdf(f"select * from {blocks2010} limit 1")

In [ ]:
river_segs = "pollution_demographics.river_segments"
engine.execute_returning_gdf(f"select * from {river_segs}")

In [ ]:
def river_blocks(name, buffer_in_miles):
  buffer_in_meters = buffer_in_miles * 1609.34
  df = engine.execute_returning_df(f"""
  create or replace function pg_temp.geoid10f ()
  returns table (txt text)
  language plpgsql
  as
  $$
  declare
    river geometry;
  begin
    select st_union(geom) into river from {river_segs} where name='{name}';
    river := st_buffer(river::geography, {buffer_in_meters})::geometry;
    return query 
      select geoid10 from {blocks2010} where st_intersects(geom, river) and (st_area(st_intersection(geom, river))/st_area(geom)) > 0.5;
  end;
  $$;

  select pg_temp.geoid10f();
  """)
  
  ret =  list(df.iloc[:, 0])
  print(f"river_blocks('{name}', {buffer_in_miles}) returns {len(ret)} blocks)")

  return ret


buffer_in_miles = 1

cancer_alley = river_blocks('Cancer_Alley', buffer_in_miles)
allegheny_to_harrison = river_blocks('Allegheny_to_Harrison', buffer_in_miles)
mon_to_wheeling = river_blocks('Mon_River_to_Wheeling', buffer_in_miles)
ohio_river = river_blocks('Ohio_River', buffer_in_miles)

buffer_in_miles = 2

cancer_alley_2 = river_blocks('Cancer_Alley', buffer_in_miles)
allegheny_to_harrison_2 = river_blocks('Allegheny_to_Harrison', buffer_in_miles)
mon_to_wheeling_2 = river_blocks('Mon_River_to_Wheeling', buffer_in_miles)
ohio_river_2 = river_blocks('Ohio_River', buffer_in_miles)

In [ ]:
import json
block_geoids_2010 = json.load(open('/projects/earthtime/census/block_geoids_2010.json'))

geoid2idx = {}
for i in range(0, len(block_geoids_2010)):
    geoid2idx[block_geoids_2010[i]] = i+1

In [ ]:
def convert_geoids_to_block_vector_and_write(geoids, filename):
    col = np.zeros(len(block_geoids_2010)+1, dtype=np.float32)
    for geoid in geoids:
        col[geoid2idx[geoid]] = 1.0
    col.tofile(filename)
    print(f"Put {len(geoids)} 1s into vector, with sum {col.sum()}.  Wrote to {filename}, with length {os.path.getsize(filename)}")

    return 

convert_geoids_to_block_vector_and_write(cancer_alley, "columns/cancer_alley_1m.float32")
convert_geoids_to_block_vector_and_write(allegheny_to_harrison, "columns/allegheny_to_harrison_1m.float32")
convert_geoids_to_block_vector_and_write(mon_to_wheeling, "columns/mon_to_wheeling_1m.float32")
convert_geoids_to_block_vector_and_write(ohio_river, "columns/ohio_river_1m.float32")
convert_geoids_to_block_vector_and_write(set(cancer_alley)|set(allegheny_to_harrison)|set(mon_to_wheeling)|set(ohio_river), "columns/river_valleys_1m.float32")

convert_geoids_to_block_vector_and_write(cancer_alley_2, "columns/cancer_alley_2m.float32")
convert_geoids_to_block_vector_and_write(allegheny_to_harrison_2, "columns/allegheny_to_harrison_2m.float32")
convert_geoids_to_block_vector_and_write(mon_to_wheeling_2, "columns/mon_to_wheeling_2m.float32")
convert_geoids_to_block_vector_and_write(ohio_river_2, "columns/ohio_river_2m.float32")
convert_geoids_to_block_vector_and_write(set(cancer_alley_2)|set(allegheny_to_harrison_2)|set(mon_to_wheeling_2)|set(ohio_river_2), "columns/river_valleys_2m.float32")


In [ ]:
!rsync -av columns/ hal15.andrew.cmu.edu:uwsgi/dotmaptiles-data/server/columncache/air_demographics

In [ ]:
engine.execute_returning_gdf("select st_area(geom) from pollution_demographics.river_segments where name='Ohio_River'")

In [ ]:
#engine.execute_returning_gdf(f"select * from {blocks2010} where geom within ")